# Работа со строковыми значениями

__Автор задач: Блохин Н.В. (NVBlokhin@fa.ru)__

Материалы:
* Макрушин С.В. Лекция "Работа со строковыми значениям"
* https://pyformat.info/
* https://docs.python.org/3/library/re.html
    * https://docs.python.org/3/library/re.html#flags
    * https://docs.python.org/3/library/re.html#functions
* https://pythonru.com/primery/primery-primeneniya-regulyarnyh-vyrazheniy-v-python
* https://kanoki.org/2019/11/12/how-to-use-regex-in-pandas/
* https://realpython.com/nltk-nlp-python/

## Задачи для совместного разбора

1. Вывести на экран данные из словаря `obj` построчно в виде `k = v`, задав формат таким образом, чтобы знак равенства оказался на одной и той же позиции во всех строках. Строковые литералы обернуть в кавычки.

In [5]:
obj = {
    "home_page": "https://github.com/pypa/sampleproject",
    "keywords": "sample setuptools development",
    "license": "MIT",
}

2. Написать регулярное выражение,которое позволит найти номера групп студентов.

In [6]:
obj = pd.Series(["Евгения гр.ПМ19-1", "Илья пм 20-4", "Анна 20-3"])
obj

NameError: name 'pd' is not defined

3. Разбейте текст формулировки задачи 2 на слова.

## Лабораторная работа 6

### Форматирование строк

In [7]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup


1\. Загрузите данные из файла `recipes_sample.csv` (__ЛР2__) в виде `pd.DataFrame` `recipes` При помощи форматирования строк выведите информацию об id рецепта и времени выполнения 5 случайных рецептов в виде таблицы следующего вида:

    
    |      id      |  minutes  |
    |--------------------------|
    |    61178     |    65     |
    |    202352    |    80     |
    |    364322    |    150    |
    |    26177     |    20     |
    |    224785    |    35     |
    
Обратите внимание, что ширина столбцов заранее неизвестна и должна рассчитываться динамически, в зависимости от тех данных, которые были выбраны. 

In [8]:
recipes_sample = pd.read_csv('data/recipes_sample.csv')

data = [recipes_sample.iloc[i] for i in np.random.randint(0, len(recipes_sample), 5)]

def f(a, b):
    return f'|{a: ^15}|{b: ^15}|'
print(f('id', 'minutes'))
print(f'|{"-"*31}|')
for i in data:
    print(f(i['id'], i['minutes']))

    

|      id       |    minutes    |
|-------------------------------|
|    193732     |      15       |
|    113913     |      35       |
|     70886     |      52       |
|    279600     |      73       |
|    490865     |      10       |


2\. Напишите функцию `show_info`, которая по данным о рецепте создает строку (в смысле объекта python) с описанием следующего вида:

```
"Название Из Нескольких Слов"

1. Шаг 1
2. Шаг 2
----------
Автор: contributor_id
Среднее время приготовления: minutes минут
```

    
Данные для создания строки получите из файлов `recipes_sample.csv` (__ЛР2__) и `steps_sample.xml` (__ЛР3__). 
Вызовите данную функцию для рецепта с id `170895` и выведите (через `print`) полученную строку на экран.

In [9]:
data = recipes_sample.loc[recipes_sample['id'] == 170895]

name = data['name'].values[0]
author = data['contributor_id'].values[0]
minutes = data['minutes'].values[0]

with open('data/steps_sample_1.xml') as f:
    dom_tree = BeautifulSoup(f, 'xml')
    recipe = dom_tree.find('id', string='67664').find_parent('recipe')
    
    steps = [[i.get_text() for i in recipe.steps][i] for i in range(1, len(recipe.steps), 2)]
  

print(f'{name}\n\n1. {steps[0]}\n2. {steps[1]}\n3. {steps[2]}\n{"-" * 10}\nАвтор:  {author}\nСреднее время приготовления: {minutes} минут ')


leeks and parsnips  sauteed or creamed

1. mix all the ingredients using a blender
2. pour into popsicle molds
3. freeze and enjoy !
----------
Автор:  8377
Среднее время приготовления: 27 минут 


## Работа с регулярными выражениями

3\. Напишите регулярное выражение, которое ищет следующий паттерн в строке: число (1 цифра или более), затем пробел, затем слова: hour или hours или minute или minutes. Произведите поиск по данному регулярному выражению в каждом шаге рецепта с id 25082. Выведите на экран все непустые результаты, найденные по данному шаблону.

In [86]:
import requests
from bs4 import BeautifulSoup
import re

with open('data/steps_sample_1.xml') as f:
    ab = BeautifulSoup(f, 'xml')
    recipe = ab.find('id', string='25082').find_parent('recipe')

    steps = [[i.get_text() for i in recipe.steps][i] for i in range(1, len(recipe.steps), 2)] #нашли все этапы рецепта

for i in steps:     
    p = re.compile(r"\d+ ((hour|hours)|(minute|minutes))", re.I)
    res = p.findall(i)
    if bool(res):
        print(i)        #проверили все шаги на соответствие регулярному выражению

turn out onto a lightly floured board and knead for about 20 minutes , adding flour as nescessary to keep the dough from sticking to the board
when it has been sufficiently kneaded , cover it with a damp cloth for about 10 minutes and wash and grease the bowl lightly
let the dough rise until it springs back when you stick your finger in it , and it is about twice the size as it was before (this takes about 2 hours
when the dough has risen twice , deflate it again and cover it with the damp cloth again for about 10 minutes , then divide it into and shape it into loaves , buns , etc
bake at 400 for 20 minutes , and then turn the oven down to 350 and bake for 20-30 minutes longer , until the loaf is a lovely brown and sounds hollow when you thump it on the bottom


In [140]:
def steps_finder3000(step_id: str = None):
    with open('data/steps_sample_1.xml') as f:
        ab = BeautifulSoup(f, 'xml')
        recipe = ab.find('id', string=step_id).find_parent('recipe')
    
        steps = [[i.get_text() for i in recipe.steps][i] for i in range(1, len(recipe.steps), 2)]
        
        return steps

['proof yeast in half a cup of the water , with about a teaspoon of flour- let it sit , covered , until it becomes bubbly& active', 'combine the rest of the water with the yeast / water in a large bowl , and gradually add about half of the flour', 'stir the mixture about 100 times in the same direction , until it is well-mixed and strands are beginning to form', 'stir in the salt and oil', 'add the remaining flour half a cup at a time , stirring well , until it is too stiff to stir with a spoon', 'turn out onto a lightly floured board and knead for about 20 minutes , adding flour as nescessary to keep the dough from sticking to the board', 'the dough should be springy and nice to work with', 'when it has been sufficiently kneaded , cover it with a damp cloth for about 10 minutes and wash and grease the bowl lightly', 'after it has rested , knead the dough a few more times and place it in the bowl , covered with the damp cloth , a plastic bag , and maybe a plate', 'let the dough rise un

4\. Напишите регулярное выражение, которое ищет шаблон вида "this..., but" _в начале строки_ . Между словом "this" и частью ", but" может находиться произвольное число букв, цифр, знаков подчеркивания и пробелов. Никаких других символов вместо многоточия быть не может. Пробел между запятой и словом "but" может присутствовать или отсутствовать.

Используя строковые методы `pd.Series`, выясните, для каких рецептов данный шаблон содержится в тексте описания. Выведите на экран количество таких рецептов и 3 примера подходящих описаний (текст описания должен быть виден на экране полностью).

In [136]:
res = recipes_sample[recipes_sample['description'].str.match(r"this [^!.,]*, but ", na=False)]
print(*list(res['description'].head(3)), sep='\n\n' )
print(f'\n\nlength = {len(res)}')


this was adapted from a recipe i found on the net, but i added julienne onion to the peppers.  this is a meal in itself, or you could have a small slice with a meat dish.  for those that like to have brunch, it's a bit different to your traditional quiche recipes.  if you love cheese, you could add 1/2 cup of your favorite to the egg mixture, then pour over peppers.

this is kind of similar to some of the other versions out there, but it is the best and easiest i have found

this recipe was originally made into a 9" pie, but i sometimes enjoy the simplicity of using it as a pudding! the junket brand danish dessert also comes in a raspberry flavor, if you can find it on the grocery shelf! preparation time does not include time needed to chill.


length = 161


5\. В текстах шагов рецептов обыкновенные дроби имеют вид "a / b". Используя регулярные выражения, уберите в тексте шагов рецепта с id 72367 пробелы до и после символа дроби. Выведите на экран шаги этого рецепта после их изменения.

In [184]:
steps = steps_finder3000('72367')
for i in range(len(steps)):
    p = re.compile(r"[0-9] / [0-9]", re.I)
    res = p.findall(steps[i])
    if res:
        steps[i] = re.sub(r"( / )", "/", steps[i])

print(*steps, sep='\n' )

mix butter , flour , 1/3 c
sugar and 1-1/4 t
vanilla
press into greased 9" springform pan
mix cream cheese , 1/4 c
sugar , eggs and 1/2 t
vanilla beating until fluffy
pour over dough
combine apples , 1/3 c
sugar and cinnamon
arrange on top of cream cheese mixture and sprinkle with almonds
bake at 350 for 45-55 minutes , or until tester comes out clean


### Сегментация текста

6\. Разбейте тексты шагов рецептов на слова при помощи пакета `nltk`. Посчитайте и выведите на экран кол-во уникальных слов среди всех рецептов. Словом называется любая последовательность алфавитных символов (для проверки можно воспользоваться `str.isalpha`). При подсчете количества уникальных слов не учитывайте регистр.

In [265]:
from nltk.tokenize.toktok import ToktokTokenizer
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk import pos_tag

toktok = ToktokTokenizer()

tokens = toktok.tokenize(' '.join(steps))

unique_tokens = set(tokens)

print(len(unique_tokens))

99


7. Разбейте описания рецептов из `recipes` на предложения при помощи пакета `nltk`. Найдите 5 самых длинных описаний (по количеству _предложений_) рецептов в датасете и выведите строки фрейма, соответствующие этим рецептами, в порядке убывания длины.

In [260]:
res=[]
for i in range(len(recipes_sample['description'])):
    if type(recipes_sample['description'][i]) == str:      
        res.append( (len(sent_tokenize(recipes_sample['description'][i])), i) )
        
for j, i in (sorted(res)[::-1][:5]):
    print(recipes_sample.iloc[i])
    print(f'sentances = {j}')

name                   my favorite buttercream icing for decorating
id                                                           334113
minutes                                                          30
contributor_id                                               681465
submitted                                                2008-10-30
n_steps                                                        12.0
description       this wonderful icing is used for icing cakes a...
n_ingredients                                                   NaN
Name: 18408, dtype: object
sentances = 76
name              alligator claws  avocado fritters  with chipot...
id                                                           287008
minutes                                                          45
contributor_id                                               765354
submitted                                                2008-02-19
n_steps                                                         NaN
descri

8. Напишите функцию, которая для заданного предложения выводит информацию о частях речи слов, входящих в предложение, в следующем виде:
```
PRP   VBD   DT      NNS     CC   VBD      NNS        RB   
 I  omitted the raspberries and added strawberries instead
``` 
Для определения части речи слова можно воспользоваться `nltk.pos_tag`.

Проверьте работоспособность функции на названии рецепта с id 241106.

Обратите внимание, что часть речи должна находиться ровно посередине над соотвествующим словом, а между самими словами должен быть ровно один пробел.


In [295]:
steps = steps_finder3000('241106')
text = ' '.join(steps)
tokens = word_tokenize(text)
tagged_tokens = pos_tag(tokens)

for i in tagged_tokens[:5]:
    print(i[1], end = ' ')

print()

for j in tagged_tokens[:5]:
    print(j[0], end= ' ')

TO VB DT JJ NN 
to make the balsamic marinade 